# 软件安装

In [99]:
!pip install pyecharts

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, http://pypi.yimian.com.cn/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [100]:
!pip install bokeh

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, http://pypi.yimian.com.cn/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


# 编程基本思路

编程绘制图形的过程，和画画相似。区别在于，我们是输入一连串的命令，让电脑进行绘制，如果画得不好，可以修改指令，让电脑重新绘制。

1. 确定图形种类
2. 数据变形（准备合适的数据）
3. 创建画布（图形操作对象）
4. 添加数据
5. 补充细节
6. 渲染图形

不同的可视化工具包，有不同的设计理念，因此实际的编程流程会有差异。

哪怕同是基于gg理论的实现，也有用户使用方便的考虑，各有区别。


## pyecharts
官方指南： https://pyecharts.org/#/zh-cn/intro

1. 创建画布（图形操作对象）
2. 添加数据
3. 补充细节
4. 渲染

度量轴的数据可以添加多次，每次对应一个实体，能很方便地绘制多实体数据的比较，比如几个店铺之间的销售数据对比

In [29]:
from example.commons import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType

# 准备数据
y_data_a = [6, 30, 60, 30, 35, 50]
y_data_b = [5, 20, 36, 10, 75, 90]
x_data = ["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"]
# 创建画布
# 可以设置一些初始化的全局参数，比如画布的长、宽，主题等等
# 可选主题：LIGHT, DARK等，具体参考 https://pyecharts.org/#/zh-cn/themes
bar = Bar(
    opts.InitOpts(
        width="800px",
        height="500px",
        theme=ThemeType.DARK
    )
)



# 绘制图形，这里是给X轴、Y轴分别加上一些数据
bar = bar.add_xaxis(x_data)\
    .add_yaxis("商家A", y_data_a)\
    .add_yaxis("商家B", y_data_b)

# 调整图形，添加标题，改变图例位置等等
title = opts.TitleOpts(
    title="Bar-基本示例", 
    subtitle="我是副标题",
    pos_left='center'
)
legend = opts.LegendOpts(
    pos_top = "bottom"
)
bar.set_global_opts(
        title_opts=title,
        legend_opts=legend
    )

    
# 在Notebook上输出图形
bar.render_notebook()

In [30]:
# 输出HTML网页
bar.render('bar_pyecharts.html')
# 输出图片
# 需要一些前置的软件，参考文档 https://pyecharts.org/#/zh-cn/render_images

'/Users/lanxin/Projects/jupyter/python教程/第四讲/bar_pyecharts.html'

## Bokeh

官方指南 https://bokeh.pydata.org/en/latest/docs/user_guide.html

1. 决定输出方式
2. 准备数据
3. 创建画布（figure)
4. 绘制图形
5. 补充细节
6. show or save


与pyecharts不同，如果有多个实体的数据需要进行对比，需要预先对数据进行扁平化处理

In [2]:
from bokeh.io import reset_output, output_notebook, output_file, show, save

reset_output()
output_notebook()


Loading BokehJS ...

In [35]:
counts

(6, 5, 30, 20, 60, 36, 30, 10, 35, 75, 50, 90)

In [5]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
import pandas as pd
from bokeh.models import ColumnDataSource, FactorRange, LabelSet



# 准备数据
y_data_a = [6, 30, 60, 30, 35, 50]
y_data_b = [5, 20, 36, 10, 75, 90]
x_data = ["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"]
stores = ['商家A', '商家B']

# 混合分类和店铺得到的x轴实际数据
x_full_data = [(category,store) for category in x_data for store in stores ]
# y轴数据
counts = sum(zip(y_data_a, y_data_b), ())
# 合并数据
data = pd.DataFrame(dict(x=x_full_data, y=counts))

# 展示原始数据
display(data)

source = ColumnDataSource(data=data)


# 创建画布
p = figure(x_range=FactorRange(*x_full_data), plot_width = 800,plot_height=500, title="Bar-基本示例",
           toolbar_location=None, tools="")

# 绘制图形
p.vbar(x='x', top='y', width=0.9, source=source)

# 补充细节
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

# 在每个条形上方添加数字信息
labels = LabelSet(x='x', y='y',
                  text='y', level='glyph',
              x_offset=-5, y_offset=5, source=source, render_mode='canvas')

p.add_layout(labels)

# show
show(p)

,x,y
0,"(衬衫, 商家A)",6
1,"(衬衫, 商家B)",5
2,"(羊毛衫, 商家A)",30
3,"(羊毛衫, 商家B)",20
4,"(雪纺衫, 商家A)",60
5,"(雪纺衫, 商家B)",36
6,"(裤子, 商家A)",30
7,"(裤子, 商家B)",10
8,"(高跟鞋, 商家A)",35
9,"(高跟鞋, 商家B)",75


In [24]:
output_file("tt.html")
save(p)

'/Users/lanxin/Projects/jupyter/python教程/第四讲/tt.html'

# 常用图表

每个可视化图形，总是有一个主要的度量轴，以及若干维度轴。

维度轴以各种方式组合，就生成了不同的图表。


本次课程会讲解直角坐标系，极坐标系的常用图表，并补充一些数在商业分析中常用的，又不属于前两种坐标系的图表。

直角坐标系的图表包含pyecharts和Bokeh的编程实践，而极坐标系、特殊坐标系只有pyecharts实现。这是因为目前Bokeh没有提供原生的极坐标和特殊坐标的图表实现，自行绘制需要各种繁琐操作。本次课程内不列出，有兴趣的同学可以课后了解一下。





## 直角坐标系

X轴可为分类变量或连续变量，表示维度。
Y轴为连续变量，表示度量。

在此基础上，可以再添加若干个轴来表示度量，甚至可让Y轴也变成分类变量，构建成新的图表

* 长度轴：例子-三维图
* 面积轴：例子-气泡图
* 颜色轴：例子-热力图

                                                                                                                                                                                                                            

### 数据

In [7]:
# 基本数据
import numpy as np


def norm_pdf(x,mu=0,sigma=1):
    pdf = np.exp(-((x - mu)**2)/(2*sigma**2)) / (sigma * np.sqrt(2*np.pi))
    return pdf
con_x = np.linspace(-3,3, 30)
con_y = norm_pdf(con_x)
# x为连续变量，标准正态分布的密度曲线数据
con_data = {
    'x':con_x,
    'y':con_y,
}

# x为分类变量，衣物细分类销量数据
cat_data = {
    'x':["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"],
    'y1':[6, 30, 60, 30, 35, 50],
    'y2':[5, 20, 36, 10, 75, 90]
}

from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import transform
heat_data = data.drop('Annual', axis=1)
heat_data.Year = heat_data.Year.astype(str)
heat_data = heat_data.set_index('Year')
heat_data.columns.name = 'Month'


### 散点图 Scatter

散点图的泛用性很强，不管是连续还是分类变量都可使用。


其优点是简洁，当数据量较大时，散点图可以直接把所有的数据都放到画布上。
因此，常常是商业数据分析中的第一步，用于了解数据的整体分布情况。

给每个点加上一个面积轴，就可以变成气泡图，商业分析中常用气泡图来分析一组商品中买得最好的商品在两个指定维度上的特征

In [9]:
con_data

{'x': array([-3.        , -2.79310345, -2.5862069 , -2.37931034, -2.17241379,
        -1.96551724, -1.75862069, -1.55172414, -1.34482759, -1.13793103,
        -0.93103448, -0.72413793, -0.51724138, -0.31034483, -0.10344828,
         0.10344828,  0.31034483,  0.51724138,  0.72413793,  0.93103448,
         1.13793103,  1.34482759,  1.55172414,  1.75862069,  1.96551724,
         2.17241379,  2.37931034,  2.5862069 ,  2.79310345,  3.        ]),
 'y': array([0.00443185, 0.0080696 , 0.01407758, 0.02352957, 0.03767987,
        0.0578115 , 0.08498233, 0.11968854, 0.161505  , 0.20879924,
        0.25863147, 0.30693281, 0.34899145, 0.38018569, 0.39681333,
        0.39681333, 0.38018569, 0.34899145, 0.30693281, 0.25863147,
        0.20879924, 0.161505  , 0.11968854, 0.08498233, 0.0578115 ,
        0.03767987, 0.02352957, 0.01407758, 0.0080696 , 0.00443185])}

In [17]:
# pyecharts 散点图
from example.commons import Faker
from pyecharts import options as opts
from pyecharts.charts import Scatter

# x轴为连续变量时，需要指定xaxis_opts的type_
c = (
        Scatter()
        .add_xaxis(con_data['x'])
        .add_yaxis(
            series_name='',
            y_axis=con_data['y'],
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Scatter 示例"),
            xaxis_opts=opts.AxisOpts(type_='value'),
        )
    )

c.render_notebook()


In [20]:
# Bokeh 散点图
from bokeh.plotting import figure, output_file, show

p = figure()

# Bokeh提供多种图形绘制散点图，这里用circle，也可替换成以下内容
# asterisk()
# circle()
# circle_cross()
# circle_x()
# cross()
# dash()
# diamond()
# diamond_cross()
# inverted_triangle()
# square()
# square_cross()
# square_x()
# triangle()
# x()

p.cross(con_data['x'], con_data['y'], size=20)

p.y_range.start = 0
show(p)


### 折线图 Line/ 阶梯图 Step

把散点连起来，就构成了折线图，如果希望做成跳跃式的，就成了阶梯图

折线图能反映Y轴度量在X轴下的趋势变化，要求X轴必须是连续变量。



In [175]:
# pyecharts 折线图
from pyecharts.charts import Line
c = (
    Line()
    .add_xaxis(con_data['x'])
    .add_yaxis(
            series_name='',
            y_axis=con_data['y'],
            label_opts=opts.LabelOpts(is_show=False)
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="折线图"),
        xaxis_opts=opts.AxisOpts(type_='value')
    )
)
c.render_notebook()



In [184]:
# pyecharts 阶梯图
from pyecharts.charts import Line

c2 = (
    Line()
    .add_xaxis(con_data['x'])
    .add_yaxis(
        series_name='',
        y_axis=con_data['y'],
        label_opts=opts.LabelOpts(is_show=False),
        is_step=True
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="阶梯图"),
        xaxis_opts=opts.AxisOpts(type_='value')
    )
)
c2.render_notebook()


In [21]:
# bokeh 折线图
from bokeh.plotting import figure, output_file, show

p = figure(plot_width=800, plot_height=400)
p.line(con_data['x'], con_data['y'])

show(p)


In [22]:
# bokeh 阶梯图
from bokeh.plotting import figure, output_file, show

p = figure(plot_width=800, plot_height=400)
p.step(con_data['x'],con_data['y'],mode="center")

show(p)


### 条形图 Bar


如果X轴的变量是无序的分类变量，就无法观察趋势，只能对比不同分类的情况。

这时候散点图仍然有效，但视觉上不够直观。如果分类数量有限，比如10个以内，绘制条形图能获得更好的视觉效果

条形图通过条形的长度差异，来展示不同分类的度量值大小差异


In [24]:
from pyecharts.charts import Bar

# pyecharts 条形图
bar = Bar()
bar = (bar.add_xaxis(cat_data['x']).
       add_yaxis("商店A", cat_data['y1']).
       add_yaxis("商店B", cat_data['y2'])
      )

# 调整图形，添加标题，改变图例位置等等
title = opts.TitleOpts(
    title="Bar 条形图", 
)
bar.set_global_opts(
        title_opts=title,
    )

bar.render_notebook()

In [26]:
cat_data

{'x': ['衬衫', '羊毛衫', '雪纺衫', '裤子', '高跟鞋', '袜子'],
 'y1': [6, 30, 60, 30, 35, 50],
 'y2': [5, 20, 36, 10, 75, 90]}

In [29]:
# Bokeh 条形图
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, FactorRange, LabelSet
from bokeh.plotting import figure
from bokeh.transform import dodge


source = ColumnDataSource(data=cat_data)


# 创建画布
p = figure(x_range=cat_data['x'], plot_width = 800,plot_height=500, title="Bar-基本示例",
           toolbar_location=None, tools="")

p.vbar(x=dodge('x', -0.15, range=p.x_range), top='y1', width=0.2, source=source,
       color="#718dbf", legend='商店A')
p.vbar(x=dodge('x', 0.15, range=p.x_range), top='y2', width=0.2, source=source,
       color="#e84d60", legend='商店B')


# 补充细节调整
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

labels_x = LabelSet(x='x', y='y1',
                  text='y1', level='glyph',
              x_offset=-25, y_offset=5, source=source, render_mode='canvas')
labels_y = LabelSet(x='x', y='y2',
                  text='y2', level='glyph',
              x_offset=10, y_offset=5, source=source, render_mode='canvas')

p.add_layout(labels_x)
p.add_layout(labels_y)

# show
show(p)

### 热力图 Heat Map

如果把Y轴也设置成维度，而用颜色来表示度量，就可以得到热力图，与透视表很像，只是把数字映射成了颜色，更形象直观。

In [2]:
# 原始数据展示
heat_data

Month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
1948,4.0,4.7,4.5,4.0,3.4,3.9,3.9,3.6,3.4,2.9,3.3,3.6
1949,5.0,5.8,5.6,5.4,5.7,6.4,7.0,6.3,5.9,6.1,5.7,6.0
1950,7.6,7.9,7.1,6.0,5.3,5.6,5.3,4.1,4.0,3.3,3.8,3.9
1951,4.4,4.2,3.8,3.2,2.9,3.4,3.3,2.9,3.0,2.8,3.2,2.9
1952,3.7,3.8,3.3,3.0,2.9,3.2,3.3,3.1,2.7,2.4,2.5,2.5
1953,3.4,3.2,2.9,2.8,2.5,2.7,2.7,2.4,2.6,2.5,3.2,4.2
1954,5.7,6.3,6.4,6.1,5.7,5.7,5.7,5.4,5.3,4.6,4.9,4.8
1955,5.8,5.7,5.2,4.9,4.2,4.4,4.0,3.8,3.5,3.4,3.8,3.9
1956,4.7,4.8,4.7,4.1,4.2,4.7,4.4,3.7,3.4,3.1,3.9,4.0


In [32]:
# 转成3维堆叠结构
import pandas as pd
stack_data = pd.DataFrame(heat_data.stack(), columns=['rate']).reset_index()
stack_data

,Year,Month,rate
0,1948,Jan,4.0
1,1948,Feb,4.7
2,1948,Mar,4.5
3,1948,Apr,4.0
4,1948,May,3.4
5,1948,Jun,3.9
6,1948,Jul,3.9
7,1948,Aug,3.6
8,1948,Sep,3.4
9,1948,Oct,2.9


In [34]:
print(x)
print(y)
print(v)

['1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
['Dec', 'Nov', 'Oct', 'Sep', 'Aug', 'Jul', 'Jun', 'May', 'Apr', 'Mar', 'Feb', 'Jan']
[['1948', 'Jan', 4.0], ['1948', 'Feb', 4.7], ['1948', 'Mar', 4.5], ['1948', 'Apr', 4.0], ['1948', 'May', 3.4], ['1948', 'Jun', 3.9], ['1948', 'Jul', 3.9], ['1948', 'Aug', 3.6], ['1948', 'Sep', 3.4], ['1948', 'Oct', 2.9], ['1948', 'Nov', 3.3], ['1948', 'Dec', 3.6], ['1949', 'Jan', 5.0], ['1949', 'Feb', 5.8], ['1949', 'Mar', 5.6], ['1949', 'Apr', 5.4], ['1949', 

In [36]:
# pyecharts 热力图
import random

from example.commons import  Faker
from pyecharts import options as opts
from pyecharts.charts import HeatMap

x = list(stack_data['Year'].unique())
y = list(reversed(stack_data['Month'].unique()))
v = [list(v) for v in stack_data.values]

heat = (
    HeatMap()
    .add_xaxis(x)
    .add_yaxis("", y, v)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="HeatMap US Unemployment 1948—2016"),
        visualmap_opts=opts.VisualMapOpts(
            min_=0,
            max_=15
        ),
    )
)
heat.render_notebook()


In [37]:
# Bokeh 热力图
import pandas as pd

from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import transform


x_range = list(stack_data['Year'].unique())
y_range = list(reversed(stack_data['Month'].unique()))
source = ColumnDataSource(stack_data)

# 颜色-数值映射
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=0, high=15)


p = figure(plot_width=800, plot_height=500, title="HeatMap US Unemployment 1948—2016",
           x_range=x_range, y_range=y_range,
           toolbar_location=None, tools="", x_axis_location="above")

p.rect(x="Year", y="Month", width=1, height=1, source=source,
       line_color=None, fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

show(p)

## 极坐标系

将直角坐标系替换成极坐标系，弧度比长度更容易看出度量的差异，在维度轴的数据量较小（10个以内）时，使用极坐标系是很好的选择。



### 饼图 Pie
实际上饼图缺失了长度轴（半径R），使用颜色作为分类维度轴。用每个分类所占的弧度大小来表示度量。

In [39]:
category = ['衬衫', '毛衣', '领带', '裤子', '风衣', '高跟鞋', '袜子']
value = [120, 44, 86, 58, 120, 31, 123]

In [41]:
[list(z) for z in zip(category, value)]

[['衬衫', 120],
 ['毛衣', 44],
 ['领带', 86],
 ['裤子', 58],
 ['风衣', 120],
 ['高跟鞋', 31],
 ['袜子', 123]]

In [45]:
from example.commons import Faker
from pyecharts import options as opts
from pyecharts.charts import Page, Pie

pie = (
        Pie()
        .add("", [list(z) for z in zip(category, value)])
        .set_global_opts(title_opts=opts.TitleOpts(title="Pie Chart"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:销量 {c}"))
    )
pie.render_notebook()

In [46]:
# Bokeh 饼图
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

data = pd.DataFrame({'category':category,"value":value})

data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(data)]

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
        tools="hover", tooltips="@category: @value")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='category', source=data)
show(p)


In [47]:
data

,category,value,angle,color
0,衬衫,120,1.295502,#3182bd
1,毛衣,44,0.475017,#6baed6
2,领带,86,0.928443,#9ecae1
3,裤子,58,0.626159,#c6dbef
4,风衣,120,1.295502,#e6550d
5,高跟鞋,31,0.334671,#fd8d3c
6,袜子,123,1.327890,#fdae6b


### 极坐标-散点图 Polar-Scartter
散点图的另一种形式，作用和直角坐标系下的散点图相似

In [51]:
# pyecharts 极坐标图
from example.commons import Faker
from pyecharts import options as opts
from pyecharts.charts import Page, Polar
import random

# 数据
r = [random.randint(1, 100) for i in range(100)]
alpha = [random.randint(1, 360) for i in range(100)]
data = list(zip(r,alpha))

polar = (
    Polar()
    .add("", data, type_="scatter", label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="Polar-Scartter"))
)

polar.render_notebook()

In [49]:
data

[(30, 90),
 (61, 188),
 (44, 271),
 (20, 271),
 (89, 316),
 (39, 156),
 (67, 316),
 (35, 29),
 (71, 234),
 (83, 124),
 (30, 156),
 (48, 240),
 (50, 320),
 (42, 103),
 (67, 333),
 (21, 287),
 (63, 180),
 (92, 144),
 (87, 17),
 (39, 268),
 (50, 14),
 (60, 318),
 (86, 269),
 (69, 90),
 (48, 82),
 (20, 282),
 (19, 117),
 (43, 123),
 (98, 296),
 (58, 69),
 (39, 121),
 (92, 207),
 (86, 160),
 (40, 60),
 (96, 103),
 (63, 136),
 (69, 36),
 (84, 103),
 (28, 285),
 (62, 123),
 (43, 116),
 (67, 295),
 (2, 100),
 (62, 260),
 (89, 339),
 (76, 337),
 (2, 155),
 (79, 50),
 (28, 140),
 (62, 112),
 (36, 249),
 (31, 122),
 (55, 66),
 (52, 308),
 (31, 312),
 (15, 247),
 (96, 7),
 (75, 56),
 (38, 146),
 (83, 77),
 (13, 100),
 (22, 271),
 (7, 200),
 (37, 316),
 (64, 168),
 (42, 166),
 (30, 293),
 (14, 130),
 (26, 177),
 (27, 27),
 (2, 80),
 (45, 79),
 (50, 13),
 (6, 165),
 (17, 262),
 (92, 61),
 (38, 322),
 (50, 52),
 (31, 318),
 (92, 314),
 (5, 10),
 (74, 358),
 (45, 36),
 (100, 208),
 (13, 337),
 (72, 11

### 雷达图

当维度的取值固定时，可以使用雷达图。

雷达图相比条形图的好处在于，可以设置每个维度值对应的最值，在图形上实现归一化，方便比较不同维度值之间的度量差异
在比较多个图例之间的差异时，能很快发现在哪些维度上差异较大


In [55]:
from pyecharts import options as opts
from pyecharts.charts import Page, Radar

v1 = [[4300, 10000, 28000, 35000, 50000, 19000]]
v2 = [[5000, 14000, 28000, 31000, 42000, 21000]]

radar = (
    Radar(init_opts=opts.InitOpts(width="800px", height="500px"))
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name="销售", max_=6500),
            opts.RadarIndicatorItem(name="管理", max_=16000),
            opts.RadarIndicatorItem(name="信息技术", max_=30000),
            opts.RadarIndicatorItem(name="客服", max_=38000),
            opts.RadarIndicatorItem(name="研发", max_=52000),
            opts.RadarIndicatorItem(name="市场", max_=25000),
        ]
    )
    .add(
        series_name="预算分配",
        data=v1,
        linestyle_opts=opts.LineStyleOpts(color="#CD0000"),
    )
    .add(
        series_name="实际开销",
        data=v2,
        linestyle_opts=opts.LineStyleOpts(color="#5CACEE"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="雷达图"), legend_opts=opts.LegendOpts()
    )
)
radar.render_notebook()

## 特殊坐标系

在理解了坐标轴的含义之后，就可以尝试脱离常规坐标系的限制，用任意的轴来组合成不同的图形

### 词云
在文本分析中很常用的初级方法之一，常见于展示最近的热门词汇

* 维度轴： 字词本身的形状
* 度量轴： 面积




In [56]:
from pyecharts import options as opts
from pyecharts.charts import Page, WordCloud
from pyecharts.globals import SymbolType


words = [
    ("Sam S Club", 10000),
    ("Macys", 6181),
    ("Amy Schumer", 4386),
    ("Jurassic World", 4055),
    ("Charter Communications", 2467),
    ("Chick Fil A", 2244),
    ("Planet Fitness", 1868),
    ("Pitch Perfect", 1484),
    ("Express", 1112),
    ("Home", 865),
    ("Johnny Depp", 847),
    ("Lena Dunham", 582),
    ("Lewis Hamilton", 555),
    ("KXAN", 550),
    ("Mary Ellen Mark", 462),
    ("Farrah Abraham", 366),
    ("Rita Ora", 360),
    ("Serena Williams", 282),
    ("NCAA baseball tournament", 273),
    ("Point Break", 265),
]


word_cloud = (
    WordCloud()
    .add("", words, word_size_range=[0, 100])
    .set_global_opts(title_opts=opts.TitleOpts(title="WordCloud"))
)
word_cloud.render_notebook()

### 漏斗图
漏斗分析，最常见于用户转化流程分析，也就是研究在整个用户的业务活动流程中，哪一个环节的流失率最高，从而实现针对性的优化

* 维度轴：业务过程
* 度量轴：宽度

In [57]:
from example.commons import Faker
from pyecharts import options as opts
from pyecharts.charts import Funnel, Page

cate = ['添加购物车', '购物车结算', '核对订单', '提交订单', '支付订单', '支付成功'] 
value = [100,70,50,31,16,10]
funnel = (
    Funnel()
    .add("", [list(z) for z in zip(cate, value)])
    .set_global_opts(title_opts=opts.TitleOpts(title="Funnel-基本示例"))
)

funnel.render_notebook()